In [1]:
import spacy
nlp_en = spacy.load('en')
nlp_de = spacy.load('de')
from spacy.en import English
parser = English()
import xml.etree.ElementTree as ET


In [18]:
from spacy.en import English
parser = English()

## Load Test Data


In [246]:
test_file_csv = "textProcessing_testKaldi.csv"
test_data = None
with open(test_file_csv, 'r') as reader:
    test_data = reader.readlines()   

In [253]:
def prompt_map_test_data():
    prompt_map = {}
    for item in test_data[1:]:
        split = item.replace("\n", "").split("\t")
        prompt = split[1]
        
        if prompt in prompt_map:
            prompt_map[prompt].append({'id': split[0], "transcript": split[3]})
        else:
            arr = []
            arr.append({'id':split[0], "transcript":split[3]})
            prompt_map[prompt] = arr
            
    return prompt_map
prompt_test_map = prompt_map_test_data()

## REFERENE GRAMMAR

In [3]:
def create_prompt_response_map(gram):
    prompt_response_map = {}
    for prompt_unit in gram:
        key = None
        responses = []
        for prompt in prompt_unit:
            if prompt.tag == None:
                print("None")
            else:
                if(prompt.tag == 'prompt'):
                    key = prompt.text
                elif(prompt.tag == 'response'):
                    responses.append(prompt.text)
        prompt_response_map[key] = responses
    return prompt_response_map

def read_grammar_and_create_map(file):
    tree = ET.parse(file)
    grammar = tree.getroot()
    return create_prompt_response_map(grammar)
    
prompt_response_map = read_grammar_and_create_map('referenceGrammar.xml')

In [6]:
def get_sag_prompts(prompt_response_map):
    sag_prompts = {}
    for key in prompt_response_map:
        if "Sag" in key:
            sag_prompts[key] = prompt_response_map[key]
    return sag_prompts
sag_prompts = get_sag_prompts(prompt_response_map)

## NLP

In [19]:
def nlp_sentence(sentence):
    parsed = parser(sentence)
    lemmas = []
    words = []
    tags = []
    poss = []
    for i, token in enumerate(parsed):
        lemma = token.lemma_
        words.append(token)
        lemmas.append(lemma)
        tag = token.tag_
        pos = token.tag_
        tags.append(tag)
        poss.append(pos)
    return words, lemmas, tags, poss

In [20]:
#
sentences = sag_prompts['Sag: Ich habe keine Reservation']

In [102]:
# special case he she it berücksichtigen mit s anhängen
def extract_verb_chunk(sentence):
    words, lemmas, tags, poss = nlp_sentence(sentence)
    verbs_poss = []
    verb_pos = None
    words_poss = []
    word_pos = None
    for index in range(0, len(words)):
        tag = tags[index]
        word = words[index]
        
        if tag == 'VBP':
            if verb_pos == None and word_pos == None:
                verb_pos = []
                word_pos = []
            verb_pos.append(tag)
            word_pos.append(word)
        elif tag == 'NN' or tag == 'NNS':
            verbs_poss.append(verb_pos)
            words_poss.append(word_pos)
            verb_pos = []
            word_pos = []
        else:
            #print(str(word) + "\t" + str(tag))
            if verb_pos == None and word_pos == None:
                continue
            verb_pos.append(tag)
            word_pos.append(word) 
    if len(verb_pos) > 0:
        verbs_poss.append(verb_pos)
        words_poss.append(word_pos)
    return verbs_poss, words_poss
            
def extract_verb(poss, words):
    one_verb = 0
    verb = None
    index = None
    #for item in range(0, )
    for i in range(0, len(poss)):
        if poss[i] =='VBP':
            one_verb += 1
            index = i
    
    if one_verb == 1 and index != None:
        return words[index]
    
    return None

In [103]:
sent = "i do not"
print(extract_verb_chunk(sent))
#print(nlp_sentence(sent))

([['VBP', 'RB']], [[do, not]])


In [104]:
verbs = []
words = []
dict_verb = []
for sentence in sentences:
    #_, lemmas, tags, poss = nlp_sentence(sentence)
    #print("%s \t %s" %(sentence, tags))
    verb, word = extract_verb_chunk(sentence)
    if len(verb)  !=  1 or len(word) != 1: 
        print("emergency")
    words.append(word[0])
    verbs.append(verb[0])
    #print(sentence)

In [81]:
for i in range(0, len(verbs)):
    print(extract_verb(verbs[0], words[0]))

do
do
do
do
do
do
do
do
do
do
do
do


In [105]:
for verb in verbs:
    print(verb)

['VBP', 'RB']
['VBP', 'RB', 'VB', 'DT']
['VBP', 'RB']
['VBP', 'RB', 'VB', 'DT']
['VBP', 'RB']
['VBP', 'RB', 'VB', 'DT']
['VBP', 'RB']
['VBP', 'RB', 'VB', 'DT']
['VBP', 'RB']
['VBP', 'RB', 'VB', 'DT']
['VBP', 'RB']
['VBP', 'RB', 'VB', 'DT']


In [139]:
## TODOD Diese Mapping
verb_combination = {}
for index in range(0, len(verbs)):
    verb = extract_verb(verbs[index], words[index])
    if verb in verb_combination:
        print(verb)
    else:
        key = {}
        last_key = None
        for item in range(0, len(verbs[index])):
            print(verbs[index][item])
            key[verbs[index][item]] = {}
            last_key = key[verbs[index][item]]
            #key[item] = 
        print(last_key)
        print(words[index])
        print(key)
        #verb_combination[verb] = {key:"test", "word": {words[index]}}
        break
    print(verb)
print("______")
verb_combination

VBP
RB
{}
[do, not]
{'VBP': {}, 'RB': {}}
______


{}

In [124]:
dict_test = {"a": {"b":{"c": "Hello"}}}

In [126]:
dict_test['a']

{'b': {'c': 'Hello'}}

In [148]:
dict_test = {}
dict_test['a'] = {}

In [149]:
dict_test['b'] = {}

In [150]:
dict_test['a'] = {'b':{}}

In [154]:
dict_test['a'] = {dict_test['a'], 'c': {}}

SyntaxError: invalid syntax (<ipython-input-154-addafdcb2428>, line 1)

In [273]:
def insert(tree, key, value):
    if key:
        first, rest = key[0], key[1:]
        if first not in tree:
            tree[first] = {}
        insert(tree[first], rest, value)
    else:
        tree['words'] = value

In [274]:
verb_combination = {}
for index in range(0, len(verbs[index])):
    #verbs[index], word[index])
    verb = extract_verb(verbs[index], words[index])
    verb_verbs = []
    verb_verbs.append(str(verb))
    verb_verbs.extend(verbs[index])
    verbs[index]
    insert(verb_combination, verb_verbs, words[index])

print(verb_combination)

{'do': {'VBP': {'RB': {'words': [do, n't], 'VB': {'DT': {'words': [do, n't, have, a]}}}}}}


In [242]:
verb_combination['do']['VBP']['RB']

{'VB': {'DT': {'words': [do, n't, have, a]}}, 'words': [do, n't]}

In [243]:
sag_prompts['Sag: Ich habe keine Reservation']

['i do not',
 'i do not have a reservation',
 "i don't",
 "i don't have a reservation",
 'no i do not',
 'no i do not have a reservation',
 "no i don't",
 "no i don't have a reservation",
 'sorry i do not',
 'sorry i do not have a reservation',
 "sorry i don't",
 "sorry i don't have a reservation"]

In [272]:
thiskey = "Sag: Ich habe keine Reservation"
for item in prompt_test_map[thiskey]:
    accept = False
    if item['transcript'] not in prompt_response_map[thiskey]:
        print(item)

{'id': '3671', 'transcript': 'no i pay is there a reservation'}
{'id': '3689', 'transcript': '***  ***  do not have a reservation'}
{'id': '3769', 'transcript': '  a tomorrow at nine   '}
{'id': '3770', 'transcript': 'i have no reservation'}
{'id': '3771', 'transcript': 'i have a   room for have a      room   '}
{'id': '3773', 'transcript': "i haven't got a reservation"}
{'id': '3774', 'transcript': 'i have not a reservation'}
{'id': '3776', 'transcript': "***  i'm mamma have a reservation"}
{'id': '3777', 'transcript': '***  ***   a have a reservation'}
{'id': '3779', 'transcript': 'where no i have not have i   have no'}
{'id': '3807', 'transcript': "no i don't have a reservation any"}
{'id': '3888', 'transcript': 'no i  ***  ***  have a      ***    '}
{'id': '3986', 'transcript': "no i don't no i do not have a reservation"}
{'id': '4014', 'transcript': 'i have no reservation'}
{'id': '4015', 'transcript': "no i don't no i do not have a reservation"}
{'id': '4186', 'transcript': 'no i

In [271]:
counter = 0
with open("reference_grammar_test.csv", 'w') as writer:
    for prompt_unit in prompt_test_map:
        for sentence in prompt_test_map[prompt_unit]:
            if "***" in sentence['transcript']:
                continue
            if sentence['transcript'] not in prompt_response_map[prompt_unit]:
                counter +=1
                writer.write(prompt_unit + "\t" + sentence['transcript'] + "\n")
        
counter

461